# Preprocessing

In this notebook, we will be preprocessing the data and conducting any necessary transformations for our models.

***

# Initialization

Importing libraries and notebooks.

In [1]:
# Importing libraries
import import_ipynb

# Importing A_eda notebook
from A_eda import *

In [2]:
# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":
    
    # Checking python hash seed
    print('Make sure the following says \'0\': ', os.environ.get('PYTHONHASHSEED'))

Make sure the following says '0':  0


In [3]:
# Creating a copy to preserve original folds
import copy

original_folds = copy.deepcopy(folds)

***

## Check the structure of our fold data

Let's check the structure of our folds before proceeding to transforming our data.

In [4]:
# Check the structure of folds

# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":
    
    for i, fold in enumerate(folds):
        X_train, X_test, y_train, y_test = fold
        print(f'Fold {i + 1}')
        print(f'Feature columns in training set: {list(X_train.columns)}')
        print(f'Feature columns in testing set: {list(X_test.columns)}\n')

Fold 1
Feature columns in training set: ['age', 'gender', 'device_type', 'ad_position', 'browsing_history', 'time_of_day']
Feature columns in testing set: ['age', 'gender', 'device_type', 'ad_position', 'browsing_history', 'time_of_day']

Fold 2
Feature columns in training set: ['age', 'gender', 'device_type', 'ad_position', 'browsing_history', 'time_of_day']
Feature columns in testing set: ['age', 'gender', 'device_type', 'ad_position', 'browsing_history', 'time_of_day']

Fold 3
Feature columns in training set: ['age', 'gender', 'device_type', 'ad_position', 'browsing_history', 'time_of_day']
Feature columns in testing set: ['age', 'gender', 'device_type', 'ad_position', 'browsing_history', 'time_of_day']

Fold 4
Feature columns in training set: ['age', 'gender', 'device_type', 'ad_position', 'browsing_history', 'time_of_day']
Feature columns in testing set: ['age', 'gender', 'device_type', 'ad_position', 'browsing_history', 'time_of_day']

Fold 5
Feature columns in training set: ['ag

Everything looks good!

***

# Imputation preparation

***

## Ordinal encoding: `time_of_day`

Since `time_of_day` is an ordinal feature, we will be encoding it using `OrdinalEncoder` with the specified order.

In [5]:
# Ordinal encode
from sklearn.preprocessing import OrdinalEncoder

# `time_of_day` should be encoded ordinally
# Creating a function to encode the splits
def KFoldOrdinalEncoder(folds, columns, orders):
    '''Perform ordinal encoding on all the columns specified in "columns" 
    and their corresponding label orders in "orders" for the feature 
    sets for each fold in "folds". Return the encoders for transformation later.'''
    encoders = {}
    
    for i, fold in enumerate(folds):
        X_train, X_test, y_train, y_test = fold
        fold_encoders = {}
        
        for col, order in zip(columns, orders):
            # Initialize encoder
            ordinal_encoder = OrdinalEncoder(categories=[order])
            
            # Encode (for both train and test)
            train_encoded = ordinal_encoder.fit_transform(X_train[[col]])
            test_encoded = ordinal_encoder.transform(X_test[[col]])
            
            # Create new encoded columns
            X_train.loc[:, f'{col}_en'] = train_encoded
            X_test.loc[:, f'{col}_en'] = test_encoded
            
            fold_encoders[col] = ordinal_encoder
        
        fold[0] = X_train
        fold[1] = X_test
        
        encoders[i + 1] = fold_encoders

    return encoders

In [6]:
# Ordinally encode `time_of_day`
oe_columns = ['time_of_day']
time_orders = ['Morning', 'Afternoon', 'Night', 'Evening', np.nan]
oe_orders = []
oe_orders.append(time_orders)

ordinal_encoders = KFoldOrdinalEncoder(folds, oe_columns, oe_orders)

***

## Label encoding the rest of the categorical features

We will now have to use `LabelEncoder` to encode the rest of our categorical features.

In [7]:
# Label encode
from sklearn.preprocessing import LabelEncoder

def KFoldLabelEncoder(folds, columns):
    '''Perform label encoding on all the columns specified in "columns" for each
    fold in "folds". Return the encoders for transformation later and a dict of the 
    labels and their corresponding values for when we process NaNs later.'''
    
    encoders = {}
    fold_labels = {}
    
    for i, fold in enumerate(folds):
        X_train, X_test, y_train, y_test = fold
        
        # Initialize encoders
        fold_encoders = {col: LabelEncoder() for col in columns}
        col_labels = {}
        
        for col, encoder in fold_encoders.items():
            # Encode (for both train and test)
            train_encoded = encoder.fit_transform(X_train.loc[:, col])
            test_encoded = encoder.transform(X_test.loc[:, col])
            
            # Add labels to col_labels
            col_labels[col] = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
            
            # Create new encoded columns
            X_train.loc[:, f'{col}_en'] = train_encoded
            X_test.loc[:, f'{col}_en'] = test_encoded
            
        # Add column labels to each fold
        fold_labels[i + 1] = col_labels
        encoders[i + 1] = fold_encoders
        
        fold[0] = X_train
        fold[1] = X_test
    
    return encoders, fold_labels

In [8]:
# Label encode our categorical features
label_categoricals = copy.deepcopy(categoricals)
label_categoricals.remove('time_of_day')

label_encoders, fold_labels = KFoldLabelEncoder(folds, label_categoricals)

# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":

    # Check how our data is looking
    display(folds[0][0].head())

,age,gender,device_type,ad_position,browsing_history,time_of_day,time_of_day_en,gender_en,device_type_en,ad_position_en,browsing_history_en
0,22.0,NaN,Desktop,Top,Shopping,Afternoon,1.0,3,0,2,3
1,NaN,Male,Desktop,Top,NaN,NaN,NaN,1,0,2,5
2,41.0,Non-Binary,NaN,Side,Education,Night,2.0,2,3,1,0
3,34.0,Male,NaN,NaN,Entertainment,Evening,3.0,1,3,3,1
4,39.0,Non-Binary,NaN,NaN,Social Media,Morning,0.0,2,3,3,4


In [9]:
# Combine encoders
encoders_full = dict()

for fold, encoders in label_encoders.items():
    folds_oe = ordinal_encoders[fold]
    
    complete_dict = encoders.copy()
    complete_dict.update(folds_oe)
    
    encoders_full[fold] = complete_dict

***

## Reinstating NaNs for imputation

Before we impute, we will reinstate NaNs for our encoded columns before imputation.

In [10]:
# Reinstate NaNs
def KFoldNaNReinstater(folds, columns, fold_labels):
    '''Reinstate NaNs in every column in "columns" according to the label in "labels"
    for every fold in "folds".'''
    
    for fold, fold_no in zip(folds, fold_labels):
        X_train, X_test, y_train, y_test = fold
        labels = fold_labels[fold_no]
        
        # Unpack the labels
        for col, col_labels in labels.items():
            
            # Reinstate NaNs
            encode_col = f'{col}_en'
            X_train[encode_col] = X_train[encode_col].map(lambda x: np.nan if x == col_labels[np.nan] else x)
            X_test[encode_col] = X_test[encode_col].map(lambda x: np.nan if x == col_labels[np.nan] else x)

In [11]:
# Put back NaNs for folds
KFoldNaNReinstater(folds, categoricals, fold_labels)

# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":

    # Check how data is looking
    display(folds[0][0].head())

,age,gender,device_type,ad_position,browsing_history,time_of_day,time_of_day_en,gender_en,device_type_en,ad_position_en,browsing_history_en
0,22.0,NaN,Desktop,Top,Shopping,Afternoon,1.0,NaN,0.0,2.0,3.0
1,NaN,Male,Desktop,Top,NaN,NaN,NaN,1.0,0.0,2.0,NaN
2,41.0,Non-Binary,NaN,Side,Education,Night,2.0,2.0,NaN,1.0,0.0
3,34.0,Male,NaN,NaN,Entertainment,Evening,3.0,1.0,NaN,NaN,1.0
4,39.0,Non-Binary,NaN,NaN,Social Media,Morning,0.0,2.0,NaN,NaN,4.0


***

## MICE

With our NaNs reinstates, we can now impute using the MICE technique. We will briefly clean up the folds by removing the non-encoded columns and perform the imputation.

In [12]:
# Cleaning up the data before imputation
def KFoldSimplifier(folds, drop_columns):
    '''Remove the specified columns in "drop_columns" for each of the folds in "folds".'''
    for fold in folds:
        X_train, X_test, y_train, y_test = fold
        
        # Drop the columns
        X_train.drop(columns=drop_columns, inplace=True)
        
        X_test_cols = list(X_test.columns)
        drop_test_columns = list(set(drop_columns).intersection(set(X_test_cols)))
        X_test.drop(columns=drop_test_columns, inplace=True)

In [13]:
# Remove non-encoded columns
drop_cols = copy.deepcopy(categoricals)

KFoldSimplifier(folds, drop_cols)

# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":

    print(drop_cols)
    
    # Check how data is looking
    display(folds[0][0].head())

['gender', 'device_type', 'ad_position', 'browsing_history', 'time_of_day']


,age,time_of_day_en,gender_en,device_type_en,ad_position_en,browsing_history_en
0,22.0,1.0,NaN,0.0,2.0,3.0
1,NaN,NaN,1.0,0.0,2.0,NaN
2,41.0,2.0,2.0,NaN,1.0,0.0
3,34.0,3.0,1.0,NaN,NaN,1.0
4,39.0,0.0,2.0,NaN,NaN,4.0


In [14]:
# Impute function
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

def KFoldMICE(folds):
    '''Perform MICE imputation for each fold in "folds".'''
    for i, fold in enumerate(folds):
        # Unpack fold
        X_train, X_test, y_train, y_test = fold
        train_columns = X_train.columns
        test_columns = X_test.columns
        
        rf_regressor = RandomForestRegressor(n_estimators=100)
        imputer = IterativeImputer(estimator=rf_regressor, max_iter=50, missing_values=np.nan)
        
        # Impute the features in the fold
        train_imputed = imputer.fit_transform(X_train)
        test_imputed = imputer.transform(X_test)
        
        train_imputed = np.round(train_imputed).astype('int')
        test_imputed = np.round(test_imputed).astype('int')
        
        # Convert back to dataframe
        train_imputed_df = pd.DataFrame(data=train_imputed, columns=train_columns)
        test_imputed_df = pd.DataFrame(data=test_imputed, columns=test_columns)
        
        fold[0] = train_imputed_df
        fold[1] = test_imputed_df

In [ ]:
# Impute
# KFoldMICE(folds)

/opt/anaconda3/envs/dsci/lib/python3.12/site-packages/sklearn/impute/_iterative.py:825: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/opt/anaconda3/envs/dsci/lib/python3.12/site-packages/sklearn/impute/_iterative.py:825: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/opt/anaconda3/envs/dsci/lib/python3.12/site-packages/sklearn/impute/_iterative.py:825: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/opt/anaconda3/envs/dsci/lib/python3.12/site-packages/sklearn/impute/_iterative.py:825: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/opt/anaconda3/envs/dsci/lib/python3.12/site-packages/sklearn/impute/_iterative.py:825: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [16]:
# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":

    # Check how data is looking
    # The training set of the first fold
    display(folds[0][0].head())

,age,time_of_day_en,gender_en,device_type_en,ad_position_en,browsing_history_en
0,22,1,1,0,2,3
1,41,1,1,0,2,2
2,41,2,2,1,1,0
3,34,3,1,1,2,1
4,39,0,2,1,0,4


In [19]:
# Checking remaining Na's

# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":
    # Checking remaining Na's
    for i, fold in enumerate(folds):
        print(f'Fold {i + 1}')
        X_train, X_test, y_train, y_test = fold
        print('Still missing in train set:\n', (X_train == -1).sum())
        print('\nStill missing in test set:\n', (X_test== -1).sum(), '\n')

Fold 1
Still missing in train set:
 age                    0
time_of_day_en         0
gender_en              0
device_type_en         0
ad_position_en         0
browsing_history_en    0
dtype: int64

Still missing in test set:
 age                    0
time_of_day_en         0
gender_en              0
device_type_en         0
ad_position_en         0
browsing_history_en    0
dtype: int64 

Fold 2
Still missing in train set:
 age                    0
time_of_day_en         0
gender_en              0
device_type_en         0
ad_position_en         0
browsing_history_en    0
dtype: int64

Still missing in test set:
 age                    0
time_of_day_en         0
gender_en              0
device_type_en         0
ad_position_en         0
browsing_history_en    0
dtype: int64 

Fold 3
Still missing in train set:
 age                    0
time_of_day_en         0
gender_en              0
device_type_en         0
ad_position_en         0
browsing_history_en    0
dtype: int64

Still missing 

Luckily, we have no persisting NaN values! Since the imputation was computationally complex, to avoid running it a second time, we will export the data. Then, we will now transform the encoded values back (inverse transform) and perform one-hot encoding before feature selection.

***

# Exporting data + reimporting

To prevent re-running our computationally heavy imputation, we will export the data and re-import it to preserve it.

In [ ]:
# Exporting folds
# folds_path = 'data/folds'

# if not os.path.exists(folds_path):
#     os.makedirs(folds_path)
    

# for i, fold in enumerate(folds):
#     fold_path = folds_path + '/' + f'{i + 1}'
    
#     if not os.path.exists(fold_path):
#         os.makedirs(fold_path)
        
#     X_train, X_test, y_train, y_test = fold
#     X_train.to_csv(fold_path + '/' + 'X_train.csv', index=False)
#     X_test.to_csv(fold_path + '/' + 'X_test.csv', index=False)
#     y_train.to_csv(fold_path + '/' + 'y_train.csv', index=False, header=['click'])
#     y_test.to_csv(fold_path + '/' + 'y_test.csv', index=False, header=['click'])


In [21]:
# Importing folds
folds = list()
folds_path = Path('data/folds')

# Iterate over each fold directory
for fold in sorted(folds_path.iterdir()):  # Ensure directories are processed in sorted order
    if fold.is_dir():
        # Load data from each fold's files
        X_train = pd.read_csv(fold / 'X_train.csv')
        X_test = pd.read_csv(fold / 'X_test.csv')
        y_train = pd.read_csv(fold / 'y_train.csv')
        y_test = pd.read_csv(fold / 'y_test.csv')

        # Append the data as a list
        folds.append([X_train, X_test, y_train, y_test])

In [22]:
# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":

    # Check how imported data is looking
    display(folds[0][0].head())

,age,time_of_day_en,gender_en,device_type_en,ad_position_en,browsing_history_en
0,22,1,1,0,2,3
1,41,1,1,0,2,2
2,41,2,2,1,1,0
3,34,3,1,1,2,1
4,39,0,2,1,0,4


***

# Transformation 2 ways

To accomodate as many models as possible, we will be splitting transformation into 2 ways: tree-based and non tree-based.

***

## Non-tree based: One-hot encoding + Standardized scaling

We will be one-hot encoding our categorical features and scaling our numerical feature `age` by standardizing it.

### Inverse transform

In [23]:
# Inverse transform function

def KFoldInverseTransform(folds, transform_columns, encoders):
    '''A function that inverse transforms the columns in 'transform_columns' using the encoders in 'encoders'
    for each of the folds in 'folds'.'''
    # Inverse transform for each fold
    for i, fold in enumerate(folds):
        
        # Unpack folds and encoders
        X_train, X_test, y_train, y_test = fold
        column_encoders = encoders[i + 1]
        
        # Inverse transform each of the columns
        for column in transform_columns:
            encoded_col_train = X_train.loc[:, f'{column}_en']
            encoded_col_test = X_test.loc[:, f'{column}_en']
            
            column_encoder = column_encoders[column]
            
            X_train.loc[:, f'{column}'] = column_encoder.inverse_transform(encoded_col_train)
            X_test.loc[:, f'{column}'] = column_encoder.inverse_transform(encoded_col_test)
        
        fold[0] = X_train
        fold[1] = X_test

In [24]:
# Inverse transform the folds
one_hot_encoding_folds = copy.deepcopy(folds)

# Do not transform time_of_day (it is already properly encoded)
nominal_categories = categoricals.copy()
nominal_categories.remove('time_of_day')

KFoldInverseTransform(one_hot_encoding_folds, nominal_categories, encoders_full)

In [25]:
# Clean up folds
nominal_categories_en = []

for col in nominal_categories:
    nominal_categories_en.append(f'{col}_en')

KFoldSimplifier(one_hot_encoding_folds, nominal_categories_en)

In [26]:
# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":

    # Check how data is looking
    # The training set of the first fold
    display(one_hot_encoding_folds[0][0].head())
    display(folds[0][0].head())

,age,time_of_day_en,gender,device_type,ad_position,browsing_history
0,22,1,Male,Desktop,Top,Shopping
1,41,1,Male,Desktop,Top,News
2,41,2,Non-Binary,Mobile,Side,Education
3,34,3,Male,Mobile,Top,Entertainment
4,39,0,Non-Binary,Mobile,Bottom,Social Media


,age,time_of_day_en,gender_en,device_type_en,ad_position_en,browsing_history_en
0,22,1,1,0,2,3
1,41,1,1,0,2,2
2,41,2,2,1,1,0
3,34,3,1,1,2,1
4,39,0,2,1,0,4


### One-hot encoding

In [30]:
# One-hot encode

def KFoldOneHotEncoder(folds, categories):
    '''One-hot encode the columns in 'columns' for every fold in 'folds'.'''
    for fold in folds:
        X_train, X_test, y_train, y_test = fold
        
        X_train = pd.get_dummies(X_train, columns=categories).astype(int)
        X_test = pd.get_dummies(X_test, columns=categories).astype(int)
        
        fold[0] = X_train
        fold[1] = X_test

In [ ]:
KFoldOneHotEncoder(one_hot_encoding_folds, nominal_categories)

In [33]:
# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":

    # Check how data is looking
    # The training set of the first fold
    display(one_hot_encoding_folds[0][0].head())
    display(folds[0][0].head())

,age,time_of_day_en,gender_Female,gender_Male,gender_Non-Binary,device_type_Desktop,device_type_Mobile,device_type_Tablet,ad_position_Bottom,ad_position_Side,ad_position_Top,browsing_history_Education,browsing_history_Entertainment,browsing_history_News,browsing_history_Shopping,browsing_history_Social Media
0,22,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0
1,41,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0
2,41,2,0,0,1,0,1,0,0,1,0,1,0,0,0,0
3,34,3,0,1,0,0,1,0,0,0,1,0,1,0,0,0
4,39,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1


,age,time_of_day_en,gender_en,device_type_en,ad_position_en,browsing_history_en
0,22,1,1,0,2,3
1,41,1,1,0,2,2
2,41,2,2,1,1,0
3,34,3,1,1,2,1
4,39,0,2,1,0,4


### Standardized scaling

In [34]:
from sklearn.preprocessing import StandardScaler

def KFoldStandardScaler(folds, columns):
    '''A function that standardizes the columns in 'columns' for each of the folds in 'folds'.'''
    
    for fold in folds:
        X_train, X_test, y_train, y_test = fold
        
        scaler = StandardScaler()
        X_train[columns] = scaler.fit_transform(X_train[columns])
        X_test[columns] = scaler.transform(X_test[columns])
            
        fold[0] = X_train
        fold[1] = X_test

In [35]:
# Standardize age
standardize_columns = []
standardize_columns.append('age')

KFoldStandardScaler(one_hot_encoding_folds, standardize_columns)

In [36]:
# NOTEBOOK EXCLUSIVE CODE
if __name__ == "__main__":

    # Check how data is looking
    # The training set of the first fold
    display(one_hot_encoding_folds[0][0].head())

,age,time_of_day_en,gender_Female,gender_Male,gender_Non-Binary,device_type_Desktop,device_type_Mobile,device_type_Tablet,ad_position_Bottom,ad_position_Side,ad_position_Top,browsing_history_Education,browsing_history_Entertainment,browsing_history_News,browsing_history_Shopping,browsing_history_Social Media
0,-1.750074,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0
1,-0.009741,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0
2,-0.009741,2,0,0,1,0,1,0,0,1,0,1,0,0,0,0
3,-0.650917,3,0,1,0,0,1,0,0,0,1,0,1,0,0,0
4,-0.192934,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1


In [40]:
# Exporting folds
folds_path = 'data/tree_folds'

if not os.path.exists(folds_path):
    os.makedirs(folds_path)
    

for i, fold in enumerate(one_hot_encoding_folds):
    fold_path = folds_path + '/' + f'{i + 1}'
    
    if not os.path.exists(fold_path):
        os.makedirs(fold_path)
        
    X_train, X_test, y_train, y_test = fold
    X_train.to_csv(fold_path + '/' + 'X_train.csv', index=False)
    X_test.to_csv(fold_path + '/' + 'X_test.csv', index=False)
    y_train.to_csv(fold_path + '/' + 'y_train.csv', index=False, header=['click'])
    y_test.to_csv(fold_path + '/' + 'y_test.csv', index=False, header=['click'])

***

# Feature selection

We will begin looking at which features will be most impactful for our model.

***

## Chi-square test

Measures the dependency between categorical features and the target.

In [41]:
# Chi-square test
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

# For every fold
for i, fold in enumerate(folds):
    X_train, X_test, y_train, y_test = fold
    
    # Get Chi-Square statistics and p-values
    chi2_stats, p_values = chi2(X_train, y_train)

    print(f'Fold {i + 1}')
    
    # Print results
    for i in range(len(chi2_stats)):
        print(f'Feature {i}: Chi2 Stat = {chi2_stats[i]}, p-value = {p_values[i]}')
    
    # Apply chi2 test
    chi2_selector = SelectKBest(chi2, k=3)  # Select top 3 features
    X_selected = chi2_selector.fit_transform(X_train, y_train)
    
    print('Selected features indices:', chi2_selector.get_support(indices=True), '\n')

Fold 1
Feature 0: Chi2 Stat = 25.729128622656688, p-value = 3.9285260604769346e-07
Feature 1: Chi2 Stat = 5.130432476199297, p-value = 0.023509869109972173
Feature 2: Chi2 Stat = 0.011852518053403208, p-value = 0.9133061803459148
Feature 3: Chi2 Stat = 0.0009611302665976048, p-value = 0.9752678641214614
Feature 4: Chi2 Stat = 11.127455254469776, p-value = 0.0008505920767839526
Feature 5: Chi2 Stat = 1.1352338306408027, p-value = 0.28666170046995965
Selected features indices: [0 1 4] 

Fold 2
Feature 0: Chi2 Stat = 77.13117902689848, p-value = 1.5997184765512533e-18
Feature 1: Chi2 Stat = 0.5256784129745833, p-value = 0.4684294327391906
Feature 2: Chi2 Stat = 0.7562592397038679, p-value = 0.38450172525401083
Feature 3: Chi2 Stat = 3.0284189554587533, p-value = 0.08181770675526708
Feature 4: Chi2 Stat = 12.318071744810124, p-value = 0.00044859387228061775
Feature 5: Chi2 Stat = 0.28004551405599476, p-value = 0.5966713867623489
Selected features indices: [0 3 4] 

Fold 3
Feature 0: Chi2 S

While we would usually choose around 5 of the best features, given our low number of features, I decided to narrow it down to the best 3. It appears that features 0, 1, 3, and 4 (`age`, `time_of_day`, `device_type`, and `ad_position` respectively) are the best features.

***

# Next steps

That concludes our preprocessing for now. In the next section, we will be trying different models and evaluating their performance.